<a href="https://colab.research.google.com/github/manuel-alvarez/scheduling/blob/master/poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schedule Generator with Genetic Algorithms

This POC is intended to be a schedule generator for schools.

There are three type of resources:
 - Classrooms
 - Teachers
 - Subjects

For this POC we will consider that all students in a class go toguether and don't move, so they are something like attached to the classroom and therefore not considered as resources, but they could.

## POC

In this first POC, we are going to consider two of the three types of resources as constant, and one of them as variable. For example:
 - 1 classroom
 - 1 teacher
 - Multiple subjects (where number of subjects is greater than segment length)

This way, we just need to use resources of one type.


In [0]:
import numpy

In [0]:
# Population (total size of time slots)
POPULATION_SIZE = 100
# Each segment represents a day
SEGMENT = 5
# Five days, since the schedule is weekly based, that are 5 segments
INDIVIDUAL_SIZE = 25
# Resources, currently, of just one type, let's say subjects
RESOURCES = 9
# Number of individuals that pass to the next generation
SURVIVAL_RATE = .3
# Rate of elements that are going to mutate
MUTATION_RATE = .2
# Rate of genes that are going to be mutate
MUTATIONS = .4
# Number of iterations we do in order to get the best approach
STEPS = 500
# Positions that must remain empty
EMPTY = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
# Severity
SEVERITY = {
    'high': 6,
    'medium': 3,
    'low': 1
}

In [0]:
def create_individual(resources_size, individual_size):
  individual = numpy.random.randint(resources_size + 1, size=individual_size)
  full = [numpy.abs(index - 1) for index in EMPTY]
  individual = [item * full[index]  for index, item in enumerate(individual)]
  return individual  

In [48]:
create_individual(RESOURCES, INDIVIDUAL_SIZE)


[8, 9, 3, 4, 3, 9, 4, 4, 7, 5, 8, 1, 1, 1, 3, 6, 4, 0, 2, 2, 3, 0, 9, 0, 0]

In [0]:
def initialize_population(population_size, individual_size, resources_size):
  population = None
  for i in range(population_size):
    if population is None:
      population = numpy.array([create_individual(resources_size, individual_size)])
    else:
      population = numpy.append(population, [create_individual(resources_size, individual_size)], axis=0)
  return population

In [0]:
def get_fitness(the_try):
  fitness = 0
  for i in range(int(numpy.ceil(INDIVIDUAL_SIZE / SEGMENT))):
    # print(f'Evaluating', i*SEGMENT, (i+1)*SEGMENT, the_try[i*SEGMENT:(i+1)*SEGMENT], 'from', the_try)
    unique, counts = numpy.unique(the_try[i*SEGMENT:(i+1)*SEGMENT], return_counts=True)
    fitness += sum(item for item in counts if item > 1)  # Not repeated resources in same segment

  # Some positions must remain empty
  fitness += sum(EMPTY[index] * SEVERITY['high'] * item / item for index, item in enumerate(the_try) if item != 0)
  # Not empty slots in try unless it's mandatory
  fitness += sum(SEVERITY['medium'] for index, item in enumerate(the_try) if item == 0 and EMPTY[index] == 0) 
  return fitness

In [0]:
def get_selection(population):
  return numpy.array(sorted([[get_fitness(the_try), the_try] for the_try in population], key=lambda x:x[0])[:int(numpy.ceil(POPULATION_SIZE * SURVIVAL_RATE))])

In [0]:
def breed(selection):
  population = numpy.array([the_try for the_try in selection])
  i = 0
  while len(population) < POPULATION_SIZE:
    parents = population[i:i+2]
    numpy.random.shuffle(parents)
    slicery = numpy.random.randint(INDIVIDUAL_SIZE) 
    population = numpy.append(population, numpy.array([numpy.append(parents[0][slicery:], parents[1][:slicery], axis=0)]), axis=0)
    i += 1
    if i == len(selection):
      i = 0
  return population

In [0]:
def mutate(population):
  mutated_population = population.copy()
  mutants = int(numpy.ceil(POPULATION_SIZE * MUTATION_RATE))
  for i in range(mutants):
    index = numpy.random.randint(POPULATION_SIZE)
    mutant = mutated_population[index]
    num_mutations = int(numpy.ceil(INDIVIDUAL_SIZE) * MUTATIONS)
    for mutation in range(num_mutations):
      mutant[numpy.random.randint(INDIVIDUAL_SIZE)] = numpy.random.randint(1, RESOURCES)
    mutated_population[index] = mutant
  return mutated_population

In [56]:
population = initialize_population(POPULATION_SIZE, INDIVIDUAL_SIZE, RESOURCES)
for i in range(STEPS):
  print(f'Step {i}')
  selection = get_selection(population)
  print(selection[0:1])
  population = breed(selection[:,1])
  population = mutate(population)
print(numpy.reshape(population[0], (5, 5)))

Step 0
[[6.0
  array([5, 6, 8, 7, 1, 3, 1, 9, 5, 3, 2, 5, 8, 7, 4, 7, 6, 5, 7, 8, 3, 9,
       7, 0, 0])]]
Step 1
[[6.0
  array([5, 6, 8, 7, 1, 3, 1, 9, 5, 3, 2, 5, 8, 7, 4, 7, 6, 5, 7, 8, 3, 9,
       7, 0, 0])]]
Step 2
[[6.0
  array([5, 6, 8, 7, 1, 3, 1, 9, 5, 3, 2, 5, 8, 7, 4, 7, 6, 5, 7, 8, 3, 9,
       7, 0, 0])]]
Step 3
[[7.0
  array([5, 8, 2, 2, 7, 5, 5, 5, 8, 3, 7, 6, 2, 8, 4, 5, 4, 8, 9, 7, 3, 2,
       7, 0, 0])]]
Step 4
[[7.0
  array([5, 8, 2, 2, 7, 5, 5, 5, 8, 3, 7, 6, 2, 8, 4, 5, 4, 8, 9, 7, 3, 2,
       7, 0, 0])]]
Step 5
[[6.0
  array([6, 4, 9, 5, 1, 6, 5, 5, 1, 7, 1, 5, 8, 9, 2, 9, 6, 8, 4, 3, 3, 8,
       8, 0, 0])]]
Step 6
[[6.0
  array([7, 5, 4, 8, 6, 5, 2, 1, 6, 7, 3, 8, 7, 3, 4, 1, 7, 1, 8, 3, 8, 7,
       5, 0, 0])]]
Step 7
[[6.0
  array([7, 5, 4, 8, 6, 5, 2, 1, 6, 7, 3, 8, 7, 3, 4, 1, 7, 1, 8, 3, 8, 7,
       5, 0, 0])]]
Step 8
[[6.0
  array([7, 5, 4, 8, 6, 5, 2, 1, 6, 7, 3, 8, 7, 3, 4, 1, 7, 1, 8, 3, 8, 7,
       5, 0, 0])]]
Step 9
[[6.0
  array([7, 5, 4, 8, 6, 